# Monte Carlo System for simple 2x2 DiFnDif

ATE

$ Y_{it} = \beta_0 + \beta_1 \text{Post}_t + \beta_2 \text{Treatment}_i + \beta_3 (\text{Post}_t \times \text{Treatment}_i) + \sum_{k=1}^{K} \beta_{k+3} (\text{X}_i = k \times \text{Treatment}_i) + \epsilon_{it} $

Where:
- $ Y_{it} $ represents the outcome variable (e.g., wages) for individual $ i $ at time $ t $.
- $ \text{Post}_t $ is a binary variable indicating whether the observation is from the post-treatment period.
- $ \text{Treatment}_i $ is a binary variable indicating whether individual $ i $ is in the treatment group.
- $ \text{X}_i $ is a categorical variable representing a conditioning variable (e.g., education level) for individual $ i $.
- $ K $ is the total number of levels of the conditioning variable.
- $ \beta_{k+3} $ represents the coefficient for the interaction between the conditioning variable level $ k $ and the treatment indicator.
- $ \epsilon_{it} $ is the error term.

1. **Estimated Treatment Effect ($ \beta $)**:
   The treatment effect ($ \beta $) is the coefficient associated with the interaction term between the treatment indicator and the post-treatment period indicator. Mathematically, it is given by:

   $ \beta = \text{Coefficient of } (\text{Post} \times \text{Treatment}) $

2. **Standard Error ($ SE $)**:
   The standard error ($ SE $) of the treatment effect estimates how much the estimated treatment effect varies across different samples. It can be calculated as the square root of the variance of the coefficient estimate. 

3. **t-statistic ($ t $)**:
   The t-statistic ($ t $) is a measure of the signal-to-noise ratio in the estimated treatment effect. It is calculated by dividing the estimated treatment effect by its standard error. Mathematically, it can be expressed as:

   $ t = \frac{\beta}{SE} $

- data generating process fertig machen mit richtiger verteilung
- monte carlo machen
- simulationsergebnisse und true values vergleichen 

# Monte Carlo for homogenous Treatment effects

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Set a seed value for reproducibility
np.random.seed(42)

# Set the coefficients (betas)
true_beta_A = 2
true_beta_B = 3
true_beta_C = 6

# Set the number of simulations
num_simulations = 1000

# Initialize lists to store estimated coefficients
estimated_beta_A_list = []
estimated_beta_B_list = []
estimated_beta_C_list = []

# Perform the simulations
for _ in range(num_simulations):
    # Generate random binary variables A, B, and C
    A = np.random.randint(0, 2, size=num_simulations)
    B = np.random.randint(0, 2, size=num_simulations)
    C = A * B  # Interaction term of A and B

    # Generate random control variables
    control_1 = np.random.normal(0, 1, size=num_simulations)
    control_2 = np.random.normal(0, 1, size=num_simulations)

    mean_error = 0  # Mean of the error term
    std_error = 10  # Standard deviation of the error term
    error = np.random.normal(mean_error, std_error, size=num_simulations)

    # Generate normally distributed outcome variable (wage)
    mean_wage = 50  # Mean wage
    std_wage = 10  # Standard deviation of wage
    wage = (
        mean_wage
        + true_beta_A * A
        + true_beta_B * B
        + true_beta_C * C
        + control_1
        + control_2
        + error
    )

    # Create a DataFrame for the variables
    data = pd.DataFrame(
        {
            "A": A,
            "B": B,
            "C": C,
            "Control_1": control_1,
            "Control_2": control_2,
            "Wage": wage,
        },
    )

    # Create the model
    X = sm.add_constant(data[["A", "B", "C", "Control_1", "Control_2"]])
    y = data["Wage"]
    model = sm.OLS(y, X)
    results = model.fit()

    # Extract the estimated coefficients and append to the lists
    estimated_beta_A_list.append(results.params["A"])
    estimated_beta_B_list.append(results.params["B"])
    estimated_beta_C_list.append(results.params["C"])

# Calculate the average estimated coefficients
average_estimated_beta_A = np.mean(estimated_beta_A_list)
average_estimated_beta_B = np.mean(estimated_beta_B_list)
average_estimated_beta_C = np.mean(estimated_beta_C_list)

# Print the true coefficients and the average estimated coefficients
print("True Coefficients:")
print("Beta_A:", true_beta_A)
print("Beta_B:", true_beta_B)
print("Beta_C:", true_beta_C)
print("\nAverage Estimated Coefficients:")
print("Beta_A (Average Estimated):", average_estimated_beta_A)
print("Beta_B (Average Estimated):", average_estimated_beta_B)
print("Beta_C (Average Estimated):", average_estimated_beta_C)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Combine the estimated coefficients into a single DataFrame
df = pd.DataFrame(
    {
        "A": estimated_beta_A_list,
        "B": estimated_beta_B_list,
        "C": estimated_beta_C_list,
    },
)

# Plot kernel density estimates for each coefficient
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, fill=True, palette="Set1")
plt.axvline(x=true_beta_A, color="red", linestyle="--", label="True Beta A")
plt.axvline(x=true_beta_B, color="blue", linestyle="--", label="True Beta B")
plt.axvline(x=true_beta_C, color="green", linestyle="--", label="True Beta C")
plt.title("Kernel Density Plot of Estimated Coefficients")
plt.xlabel("Estimated Coefficient Value")
plt.ylabel("Density")
plt.legend()
plt.show()

# Monte Carlo for hetergenous Treatment effects 

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Set a seed value for reproducibility
np.random.seed(42)

# Set the true coefficients
true_beta_A = 2
true_beta_B = 3
true_beta_C = 6
true_beta_A_X = -20  # Interaction effect of A and covariate X
true_beta_B_X = 40  # Interaction effect of B and covariate X

# Set the number of simulations
num_simulations = 1000

# Initialize lists to store estimated coefficients for both models
estimated_beta_homogeneous_list = []
estimated_beta_heterogeneous_list = []

# Perform the simulations
for _ in range(num_simulations):
    # Generate random binary variables A, B, and C
    A = np.random.randint(0, 2, size=num_simulations)
    B = np.random.randint(0, 2, size=num_simulations)
    C = np.random.randint(0, 2, size=num_simulations)

    # Generate random covariate X
    X = np.random.normal(0, 1, size=num_simulations)

    # Generate random control variables
    control_1 = np.random.normal(0, 1, size=num_simulations)
    control_2 = np.random.normal(0, 1, size=num_simulations)

    mean_error = 0  # Mean of the error term
    std_error = 10  # Standard deviation of the error term
    error = np.random.normal(mean_error, std_error, size=num_simulations)

    # Generate normally distributed outcome variable (wage)
    mean_wage = 50  # Mean wage
    std_wage = 10  # Standard deviation of wage
    wage = (
        mean_wage
        + true_beta_A * A
        + true_beta_B * B
        + true_beta_C * C
        + true_beta_A_X * A * X
        + true_beta_B_X * B * X
        + control_1
        + control_2
        + error
    )

    # Create a DataFrame for the variables
    data = pd.DataFrame(
        {
            "A": A,
            "B": B,
            "C": C,
            "X": X,
            "Control_1": control_1,
            "Control_2": control_2,
            "Wage": wage,
        },
    )

    # Create the homogenous treatment effect model
    X_homogeneous = sm.add_constant(
        data[["A", "B", "C", "X", "Control_1", "Control_2"]],
    )
    y_homogeneous = data["Wage"]
    model_homogeneous = sm.OLS(y_homogeneous, X_homogeneous)
    results_homogeneous = model_homogeneous.fit()
    estimated_beta_homogeneous_list.append(results_homogeneous.params["C"])

    # Create the heterogeneous treatment effect model
    data["A_X"] = data["A"] * data["X"]
    data["B_X"] = data["B"] * data["X"]
    X_heterogeneous = sm.add_constant(
        data[["A", "B", "C", "X", "A_X", "B_X", "Control_1", "Control_2"]],
    )
    y_heterogeneous = data["Wage"]
    model_heterogeneous = sm.OLS(y_heterogeneous, X_heterogeneous)
    results_heterogeneous = model_heterogeneous.fit()
    estimated_beta_heterogeneous_list.append(results_heterogeneous.params["C"])

# Calculate the average estimated coefficients for both models
average_estimated_beta_homogeneous = np.mean(estimated_beta_homogeneous_list)
average_estimated_beta_heterogeneous = np.mean(estimated_beta_heterogeneous_list)

# Print the true coefficient and the average estimated coefficients for both models
print("True Coefficient (Heterogeneous Treatment Effect):", true_beta_C)
print(
    "Average Estimated Coefficient (Homogeneous Treatment Effect Model):",
    average_estimated_beta_homogeneous,
)
print(
    "Average Estimated Coefficient (Heterogeneous Treatment Effect Model):",
    average_estimated_beta_heterogeneous,
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define custom colors for coefficients
homogeneous_color = "blue"
heterogeneous_color = "orange"

# Combine the estimated coefficients into DataFrames
homogeneous_df = pd.DataFrame(
    {"Coefficient": estimated_beta_homogeneous_list, "Model": "Homogeneous"},
)
heterogeneous_df = pd.DataFrame(
    {"Coefficient": estimated_beta_heterogeneous_list, "Model": "Heterogeneous"},
)

# Concatenate DataFrames
df = pd.concat([homogeneous_df, heterogeneous_df])

# Plot kernel density estimates for each coefficient
plt.figure(figsize=(10, 6))
sns.kdeplot(
    data=df,
    x="Coefficient",
    hue="Model",
    fill=True,
    palette={"Homogeneous": homogeneous_color, "Heterogeneous": heterogeneous_color},
)
plt.axvline(x=true_beta_C, color="green", linestyle="--", label="True Beta C")
plt.title("Kernel Density Plot of Estimated Coefficients")
plt.xlabel("Estimated Coefficient Value")
plt.ylabel("Density")
plt.legend()
plt.show()

# Monte Carlo for Deep Learning and propensity scores

# Monte Carlo for 2xt DifDif

# Latest system for difference in difference estimation

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
N = 1000  # Number of simulations
T = 10  # Number of observations per group
beta = 2.0  # True treatment effect
gamma = 0.5  # True time trend coefficient
sigma = 1.0  # Standard deviation of error term

# Initialize arrays to store results
bias = []
rmse = []

In [ ]:
# Generate data
pre_treatment = np.random.normal(0, sigma, (T, 2))  # Two groups: treatment and control
time_trend = np.arange(1, T + 1) * gamma
post_treatment = pre_treatment.copy()
post_treatment[:, 0] += beta  # Introduce treatment effect for treatment group
post_treatment[:, :] += time_trend.reshape(-1, 1)  # Add time trend

# Combine pre and post treatment data
data = np.vstack((pre_treatment, post_treatment))
groups = np.repeat(["Control", "Treatment"], T)
time_periods = np.repeat(np.arange(1, T + 1), 2)

df = pd.DataFrame({"Group": groups, "Time": time_periods, "Outcome": data.flatten()})

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Define parameters
N = 1000  # Number of simulations
T = 10  # Number of observations per group
beta = 2.0  # True treatment effect
gamma = 0.5  # True time trend coefficient
sigma = 1.0  # Standard deviation of error term

# Initialize arrays to store results
bias = []
rmse = []

# Run simulations
for _ in range(N):
    # Generate data
    pre_treatment = np.random.normal(
        0,
        sigma,
        (T, 2),
    )  # Two groups: treatment and control
    time_trend = np.arange(1, T + 1) * gamma
    post_treatment = pre_treatment.copy()
    post_treatment[:, 0] += beta  # Introduce treatment effect for treatment group
    post_treatment[:, :] += time_trend.reshape(-1, 1)  # Add time trend

    # Combine pre and post treatment data
    data = np.vstack((pre_treatment, post_treatment))
    groups = np.repeat(["Control", "Treatment"], T)
    time_periods = np.repeat(np.arange(1, T + 1), 2)

    df = pd.DataFrame(
        {"Group": groups, "Time": time_periods, "Outcome": data.flatten()},
    )

    # Run DiD regression
    df["Treat"] = (df["Group"] == "Treatment").astype(int)
    df["Post"] = (df["Time"] > T).astype(int)
    df["Treat_Post"] = df["Treat"] * df["Post"]

    X = df[["Treat", "Post", "Treat_Post"]]
    X = sm.add_constant(X)
    y = df["Outcome"]

    model = sm.OLS(y, X).fit()

    # Extract estimated treatment effect
    est_beta = model.params["Treat_Post"]

    # Calculate bias and RMSE
    bias.append(est_beta - beta)
    rmse.append((est_beta - beta) ** 2)

# Calculate mean bias and RMSE
mean_bias = np.mean(bias)
mean_rmse = np.sqrt(np.mean(rmse))

print("Mean Bias:", mean_bias)
print("RMSE:", mean_rmse)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# Define function to generate synthetic data with known true ATE


def generate_data_with_true_ate(n_samples, true_ate, seed):
    np.random.seed(seed)

    control_data = pd.DataFrame(
        {
            "individual": range(1, n_samples + 1),
            "Age": np.random.randint(20, 65, size=n_samples),
            "WagePartner_income": np.random.normal(
                loc=30000,
                scale=5000,
                size=n_samples,
            ),
            "education_level": np.random.choice(
                ["No High School", "High School", "Bachelor", "Master", "PhD"],
                size=n_samples,
            ),
            "time": np.random.choice([-2, -1, 1, 2, 3, 4, 5, 6, 7, 8], size=n_samples),
        },
    )

    fq_levels = ["Low", "High"]
    reform_levels = ["Before", "After"]
    categorical_data = pd.DataFrame(
        {
            "individual": range(1, n_samples + 1),
            "FQ": np.random.choice(fq_levels, size=n_samples),
            "Reform": np.random.choice(reform_levels, size=n_samples),
        },
    )

    fq_encoding = {"Low": 0, "High": 1}
    reform_encoding = {"Before": 0, "After": 1}
    categorical_data["FQ_encoded"] = categorical_data["FQ"].map(fq_encoding)
    categorical_data["Reform_encoded"] = categorical_data["Reform"].map(reform_encoding)

    categorical_data["interaction_effect"] = (
        categorical_data["FQ_encoded"] * categorical_data["Reform_encoded"]
    )

    control_data["wage_year_male"] = np.random.normal(
        loc=30,
        scale=10,
        size=n_samples,
    ) * (1 + 0.1 * categorical_data["interaction_effect"])
    control_data["wage_year_female"] = control_data["WagePartner_income"]
    control_data["dependent_variable"] = (
        control_data["wage_year_male"] - control_data["wage_year_female"]
    )

    education_encoding = {
        "No High School": 0,
        "High School": 1,
        "Bachelor": 2,
        "Master": 3,
        "PhD": 4,
    }
    control_data["education_level_encoded"] = control_data["education_level"].map(
        education_encoding,
    )

    data = pd.merge(control_data, categorical_data, on="individual")
    data = data.drop(["Reform", "FQ", "education_level"], axis=1)
    data = data.drop(
        ["WagePartner_income", "wage_year_male", "wage_year_female"],
        axis=1,
    )

    return data, true_ate

In [ ]:
# Function to compute Difference-in-Differences (DiD) with known true ATE


def difference_in_differences_known_ate(data):
    treatment_group = data[data["Reform_encoded"] == 1]
    control_group = data[data["Reform_encoded"] == 0]

    before_treatment_treatment_group = treatment_group[treatment_group["time"] < 0][
        "dependent_variable"
    ].mean()
    after_treatment_treatment_group = treatment_group[treatment_group["time"] > 0][
        "dependent_variable"
    ].mean()
    before_treatment_control_group = control_group[control_group["time"] < 0][
        "dependent_variable"
    ].mean()
    after_treatment_control_group = control_group[control_group["time"] > 0][
        "dependent_variable"
    ].mean()

    pre_treatment_difference = (
        before_treatment_treatment_group - before_treatment_control_group
    )
    post_treatment_difference = (
        after_treatment_treatment_group - after_treatment_control_group
    )

    return post_treatment_difference - pre_treatment_difference

In [ ]:
# Run Monte Carlo simulation with known true ATE
num_simulations = 1000
ate_results_with_true_ate = []

seed_value = 634

true_ate = 10  # Set true ATE value

In [ ]:
n_samples = 100
generate_data_with_true_ate(n_samples, true_ate, seed_value)

In [ ]:
for i in range(num_simulations):
    seed = seed_value + i
    synthetic_data, true_ate = generate_data_with_true_ate(
        n_samples=1000,  # sample size
        true_ate=true_ate,
        seed=seed,
    )
    ate = difference_in_differences_known_ate(synthetic_data)

# Calculate mean and standard error of estimated ATE

# Calculate mean and standard error of estimated ATE
ate_results_with_true_ate = np.array(ate_results_with_true_ate)
mean_ate = np.mean(ate)
std_err_ate = np.std(ate, ddof=1) / np.sqrt(
    num_simulations,
)

# Calculate t-value
t_value_ate = mean_ate / std_err_ate

# Calculate p-value
degrees_of_freedom = num_simulations - 1
p_value_sim_ate = stats.t.cdf(
    t_value_ate,
    df=degrees_of_freedom,
)  # richtige verteilung?
critical_value_simulated = stats.t.ppf((1 + 0.95) / 2, df=degrees_of_freedom)

# Calculate confidence interval for simulated ATE

mean_ate_simulated = np.mean(ate)
std_err_ate_simulated = np.std(ate, ddof=1) / np.sqrt(
    num_simulations,
)
margin_of_error_simulated = critical_value_simulated * std_err_ate_simulated
lower_bound_simulated = mean_ate_simulated - margin_of_error_simulated
upper_bound_simulated = mean_ate_simulated + margin_of_error_simulated


# Calculate confidence interval for true ATE
critical_value_with_true_ate = stats.t.ppf((1 + 0.95) / 2, df=degrees_of_freedom)
margin_of_error_with_true_ate = critical_value_with_true_ate * std_err_ate
lower_bound_with_true_ate = true_ate - margin_of_error_with_true_ate
upper_bound_with_true_ate = true_ate + margin_of_error_with_true_ate

In [ ]:
# Print summary statistics

# Print summary statistics
print("Mean ATE with known true ATE:", 10)
print(
    "Confidence Interval for simulated ATE:",
    (lower_bound_simulated, upper_bound_simulated),
)
print("Mean ATE of simulated ATE:", mean_ate)
print("Standard Error of simulated ATE:", std_err_ate)
print("t-value of simulated ATE:", t_value_ate)

In [ ]:
plt.figure(figsize=(14, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(
    ate_results_with_true_ate[:, 1],
    bins=30,
    color="skyblue",
    edgecolor="black",
    alpha=0.7,
    label="Simulated ATE",
)
plt.axvline(
    mean_ate_with_true_ate,
    color="red",
    linestyle="--",
    linewidth=1.5,
    label="Mean Simulated ATE",
)
plt.axvline(true_ate, color="green", linestyle="--", linewidth=1.5, label="True ATE")
plt.xlabel("Average Treatment Effect (ATE)")
plt.ylabel("Frequency")
plt.title("Distribution of Simulated Average Treatment Effects (ATE)")
plt.legend()
plt.grid(True)

In [ ]:
# Boxplot
plt.subplot(1, 2, 2)
plt.boxplot(
    [ate_results_with_true_ate[:, 1], [true_ate]],
    labels=["Simulated ATE", "True ATE"],
)
plt.ylabel("Average Treatment Effect (ATE)")
plt.title("Boxplot of Simulated and True Average Treatment Effects (ATE)")

plt.tight_layout()
plt.show()

CTE

\[ Y_{it} = \beta_0 + \beta_1 \text{Post}_t + \beta_2 \text{Treatment}_i + \beta_3 (\text{Post}_t \times \text{Treatment}_i) + \delta (\text{Post}_t \times \text{Treatment}_i) + \epsilon_{it} \]

Where:
- \( Y_{it} \) represents the outcome variable (e.g., wages) for individual \( i \) at time \( t \).
- \( \text{Post}_t \) is a binary variable indicating whether the observation is from the post-treatment period.
- \( \text{Treatment}_i \) is a binary variable indicating whether individual \( i \) is in the treatment group.
- \( \delta \) represents the coefficient for the interaction between the post-treatment period and the treatment group, capturing the average treatment effect (ATE).
- \( \epsilon_{it} \) is the error term.


ATE

\[ Y_{it} = \beta_0 + \beta_1 \text{Post}_t + \beta_2 \text{Treatment}_i + \beta_3 (\text{Post}_t \times \text{Treatment}_i) + \sum_{k=1}^{K} \beta_{k+3} (\text{X}_i = k \times \text{Treatment}_i) + \epsilon_{it} \]

Where:
- \( Y_{it} \) represents the outcome variable (e.g., wages) for individual \( i \) at time \( t \).
- \( \text{Post}_t \) is a binary variable indicating whether the observation is from the post-treatment period.
- \( \text{Treatment}_i \) is a binary variable indicating whether individual \( i \) is in the treatment group.
- \( \text{X}_i \) is a categorical variable representing a conditioning variable (e.g., education level) for individual \( i \).
- \( K \) is the total number of levels of the conditioning variable.
- \( \beta_{k+3} \) represents the coefficient for the interaction between the conditioning variable level \( k \) and the treatment indicator.
- \( \epsilon_{it} \) is the error term.


## the modell converges very fast to the true value

- proposed changes: more zeroes in the income values 
- CATE and estimation with a non CATE DiD model

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats


def generate_data_with_true_ate(n_samples, true_ate, seed):
    np.random.seed(seed)

    control_data = pd.DataFrame(
        {
            "individual": range(1, n_samples + 1),
            "Age": np.random.randint(20, 65, size=n_samples),
            "WagePartner_income": np.random.normal(
                loc=30000,
                scale=5000,
                size=n_samples,
            ),
            "education_level": np.random.choice(
                ["No High School", "High School", "Bachelor", "Master", "PhD"],
                size=n_samples,
            ),
            "time": np.random.choice([-2, -1, 1, 2, 3, 4, 5, 6, 7, 8], size=n_samples),
        },
    )

    fq_levels = ["Low", "High"]
    reform_levels = ["Before", "After"]
    categorical_data = pd.DataFrame(
        {
            "individual": range(1, n_samples + 1),
            "FQ": np.random.choice(fq_levels, size=n_samples),
            "Reform": np.random.choice(reform_levels, size=n_samples),
        },
    )

    fq_encoding = {"Low": 0, "High": 1}
    reform_encoding = {"Before": 0, "After": 1}
    categorical_data["FQ_encoded"] = categorical_data["FQ"].map(fq_encoding)
    categorical_data["Reform_encoded"] = categorical_data["Reform"].map(reform_encoding)

    categorical_data["interaction_effect"] = (
        categorical_data["FQ_encoded"] * categorical_data["Reform_encoded"]
    )

    control_data["wage_year_male"] = np.random.normal(
        loc=30,
        scale=10,
        size=n_samples,
    ) * (1 + 0.1 * categorical_data["interaction_effect"])
    control_data["wage_year_female"] = control_data["WagePartner_income"]
    control_data["dependent_variable"] = (
        control_data["wage_year_male"] - control_data["wage_year_female"]
    )

    education_encoding = {
        "No High School": 0,
        "High School": 1,
        "Bachelor": 2,
        "Master": 3,
        "PhD": 4,
    }
    control_data["education_level_encoded"] = control_data["education_level"].map(
        education_encoding,
    )

    data = pd.merge(control_data, categorical_data, on="individual")
    data = data.drop(["Reform", "FQ", "education_level"], axis=1)
    data = data.drop(
        ["WagePartner_income", "wage_year_male", "wage_year_female"],
        axis=1,
    )

    return data, true_ate


def difference_in_differences_cte(data, conditioning_variable):
    # Define treatment and control groups
    treatment_group = data[data["Reform_encoded"] == 1]
    control_group = data[data["Reform_encoded"] == 0]

    # Initialize lists to store CTEs for each level of the conditioning variable
    ctes = []

    # Compute CTE for each level of the conditioning variable
    for level in data[conditioning_variable].unique():
        treatment_group_level = treatment_group[
            treatment_group[conditioning_variable] == level
        ]
        control_group_level = control_group[
            control_group[conditioning_variable] == level
        ]

        pre_diff_treatment_group = treatment_group_level[
            treatment_group_level["time"] < 0
        ]["dependent_variable"].mean()
        post_diff_treatment_group = treatment_group_level[
            treatment_group_level["time"] > 0
        ]["dependent_variable"].mean()
        pre_diff_control_group = control_group_level[control_group_level["time"] < 0][
            "dependent_variable"
        ].mean()
        post_diff_control_group = control_group_level[control_group_level["time"] > 0][
            "dependent_variable"
        ].mean()

        cte_treatment_group = post_diff_treatment_group - pre_diff_treatment_group
        cte_control_group = post_diff_control_group - pre_diff_control_group

        cte = cte_treatment_group - cte_control_group
        ctes.append(cte)

    # Compute overall CTE
    return sum(ctes)


# Generate data
num_simulations = 1000
seed_value = 42
true_ate = 10  # Adjust as needed
ate_results_with_true_ate = []

for i in range(num_simulations):
    seed = seed_value + i
    synthetic_data, _ = generate_data_with_true_ate(
        n_samples=1000,
        true_ate=true_ate,
        seed=seed,
    )
    cte = difference_in_differences_cte(synthetic_data, "education_level_encoded")
    ate_results_with_true_ate.append(cte)

# Calculate mean and standard error of estimated CTE
mean_cte_with_true_ate = np.mean(ate_results_with_true_ate)
std_err_cte_with_true_ate = np.std(ate_results_with_true_ate, ddof=1) / np.sqrt(
    num_simulations,
)

# Calculate t-value
t_value_with_true_ate = mean_cte_with_true_ate / std_err_cte_with_true_ate

# Calculate p-value
degrees_of_freedom = num_simulations - 1
p_value_with_true_ate = stats.t.cdf(t_value_with_true_ate, df=degrees_of_freedom)

# Calculate confidence interval for simulated CTE
critical_value_with_true_ate = stats.t.ppf((1 + 0.95) / 2, df=degrees_of_freedom)
margin_of_error_with_true_ate = critical_value_with_true_ate * std_err_cte_with_true_ate
lower_bound_with_true_ate = mean_cte_with_true_ate - margin_of_error_with_true_ate
upper_bound_with_true_ate = mean_cte_with_true_ate + margin_of_error_with_true_ate

# Print summary statistics
print("True CTE:", true_ate)
print("Mean CTE with known true CTE:", mean_cte_with_true_ate)
print("Standard Error of CTE with known true CTE:", std_err_cte_with_true_ate)
print("t-value with known true CTE:", t_value_with_true_ate)
print("p-value with known true CTE:", p_value_with_true_ate)
print(
    "Confidence Interval for simulated CTE:",
    (lower_bound_with_true_ate, upper_bound_with_true_ate),
)

In [ ]:
import matplotlib.pyplot as plt

# Plot histogram of simulated CTEs
plt.hist(
    ate_results_with_true_ate,
    bins=20,
    color="skyblue",
    edgecolor="black",
    alpha=0.7,
    label="Simulated CTEs",
)
plt.axvline(
    x=mean_cte_with_true_ate,
    color="red",
    linestyle="--",
    label="Mean Simulated CTE",
)

# Plot true CTE
plt.axvline(x=true_ate, color="green", linestyle="-", label="True CTE")

plt.xlabel("Conditional Treatment Effect (CTE)")
plt.ylabel("Frequency")
plt.title("Histogram of Simulated and True Conditional Treatment Effects")
plt.legend()
plt.show()